# 04 · Classification : prédire les athlètes médaillés

Objectif : entraîner des modèles supervisés pour prédire si un athlète ou une équipe remporte une médaille à partir des caractéristiques disponibles.

## Plan du notebook
- Charger le dataset fusionné `olympic_full.csv` généré lors du préprocessing.
- Sélectionner les variables pertinentes et préparer les features (numeriques / catégorielles).
- Mettre en place un pipeline `sklearn` (imputation + encodage + scaler + modèle).
- Tester plusieurs modèles (Decision Tree, Random Forest) et réaliser une GridSearch.
- Évaluer les performances (accuracy, rapport de classification, matrice de confusion).
- Sauvegarder le meilleur modèle et les métriques pour usage futur.

In [ ]:
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
import joblib

sns.set_theme(style='whitegrid', context='talk')
np.random.seed(42)

In [ ]:
BASE_DIR = Path('..').resolve()
DATA_DIR = BASE_DIR / 'data'
PROCESSED_DIR = DATA_DIR / 'processed'
MODELS_DIR = BASE_DIR / 'models'
REPORTS_DIR = BASE_DIR / 'reports'
FIG_DIR = REPORTS_DIR / 'figures'
MODELS_DIR.mkdir(parents=True, exist_ok=True)
FIG_DIR.mkdir(parents=True, exist_ok=True)

full_data_path = PROCESSED_DIR / 'olympic_full.csv'
full_data_path

In [ ]:
if not full_data_path.exists():
    raise FileNotFoundError('olympic_full.csv introuvable. Exécuter 02_preprocessing.ipynb avant ce notebook.')

df = pd.read_csv(full_data_path)
df.head(3)

## Préparation des features
On définit la cible (`medal_flag`) et on distingue les colonnes numériques / catégorielles exploitables. On supprime les colonnes non informatives (identifiants, URLs).

In [ ]:
target_col = 'medal_flag'
drop_cols = ['athlete_url', 'athlete_full_name', 'medal_type_result', 'medal_type_medals', 'medal_type_final']
available_cols = [c for c in df.columns if c not in drop_cols and c != target_col]
X = df[available_cols].copy()
y = df[target_col].astype(int)

numeric_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_cols = X.select_dtypes(include=['object']).columns.tolist()
numeric_cols, categorical_cols

## Découpage train / test
On utilise une stratification sur la cible pour préserver le ratio médaillé/non médaillé.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)
X_train.shape, X_test.shape

## Pipeline sklearn
Imputation des valeurs manquantes, standardisation des numériques et encodage One-Hot des catégorielles.

In [ ]:
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler()),
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore')),
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_cols),
        ('cat', categorical_transformer, categorical_cols)
    ]
)
preprocessor

## Modèles de base
On compare un Decision Tree et un Random Forest sans tuning pour obtenir une baseline.

In [ ]:
models = {
    'decision_tree': DecisionTreeClassifier(random_state=42),
    'random_forest': RandomForestClassifier(random_state=42),
    'logistic_regression': LogisticRegression(max_iter=1000, random_state=42)
}

baseline_reports = {}
for name, clf in models.items():
    pipe = Pipeline([
        ('preprocess', preprocessor),
        ('clf', clf)
    ])
    pipe.fit(X_train, y_train)
    preds = pipe.predict(X_test)
    baseline_reports[name] = classification_report(y_test, preds, output_dict=True)
    print(f"\n=== {name.upper()} ===")
    print(classification_report(y_test, preds))

## GridSearch RandomForest
Affinage d'un RandomForest via validation croisée (5 folds).

In [ ]:
rf_pipeline = Pipeline([
    ('preprocess', preprocessor),
    ('clf', RandomForestClassifier(random_state=42))
])

param_grid = {
    'clf__n_estimators': [200, 400],
    'clf__max_depth': [None, 15, 30],
    'clf__min_samples_split': [2, 5],
    'clf__min_samples_leaf': [1, 3]
}

grid_search = GridSearchCV(
    rf_pipeline,
    param_grid=param_grid,
    cv=5,
    scoring='accuracy',
    n_jobs=-1,
    verbose=1
)

In [ ]:
grid_search.fit(X_train, y_train)
grid_search.best_params_, grid_search.best_score

In [ ]:
best_model = grid_search.best_estimator_
test_preds = best_model.predict(X_test)
test_report = classification_report(y_test, test_preds, output_dict=True)
print(classification_report(y_test, test_preds))

## Matrice de confusion
Visualisation pour comprendre les erreurs du modèle final.

In [ ]:
cm = confusion_matrix(y_test, test_preds)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
fig, ax = plt.subplots(figsize=(6, 5))
disp.plot(ax=ax, cmap='Blues', colorbar=False)
ax.set_title('Matrice de confusion - RandomForest optimisé')
plt.tight_layout()
fig.savefig(FIG_DIR / 'classification_confusion_matrix.png', dpi=120)
fig

## Sauvegarde du modèle et des résultats
Permet une réutilisation dans la webapp ou les scripts `src/models`.

In [ ]:
model_path = MODELS_DIR / 'rf_classifier_medal.joblib'
joblib.dump(best_model, model_path)
model_path

In [ ]:
metrics_summary = {
    'best_params': grid_search.best_params_,
    'best_cv_score': grid_search.best_score_,
    'test_accuracy': test_report['accuracy']
}
metrics_path = REPORTS_DIR / 'classification_metrics.csv'
pd.DataFrame(test_report).to_csv(metrics_path)
metrics_summary

## Pistes d'amélioration
- Tester d'autres modèles (SVM, Gradient Boosting) pour comparer les performances.
- Gérer les déséquilibres éventuels via `class_weight` ou sur-échantillonnage (SMOTE).
- Ajouter des features socio-économiques ou historiques pour enrichir la prédiction.
- Suivre l'évolution des performances par édition ou par discipline.